# This code fetches era5 monthly averaged reanalysis temperature data from cds from 1994 till 2024 for Africa and averages this 30-year temperature data to a singleband geotiff file for visualization in QGIS

In [1]:
import cdsapi

In [22]:
import numpy as np

In [2]:
#set your CDS API key and url
url = "https://cds.climate.copernicus.eu/api/v2"
cdsapi_key = "275708:61653f8b-f5e5-4f25-8400-f7cd5d1a67b1"

In [3]:
c = cdsapi.Client(url = url, key = cdsapi_key )

In [4]:
c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'grib',
        'product_type': 'monthly_averaged_reanalysis',
        'variable': '2m_temperature',
        'year': list(range(1994, 2024)),
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': '00:00',
        'area': [40, -25, -35, 60],
    },
    'download.grib')

2024-01-21 05:55:27,956 INFO Welcome to the CDS
2024-01-21 05:55:27,961 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-01-21 05:55:28,456 INFO Request is completed
2024-01-21 05:55:28,458 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data2/adaptor.mars.internal-1705773126.0350935-9805-13-9e29fb9d-f311-4830-ae7f-43c2a8c8b47c.grib to download.grib (70.5M)
2024-01-21 05:58:21,233 ERROR Download interupted: HTTPSConnectionPool(host='download-0016.copernicus-climate.eu', port=443): Read timed out.
2024-01-21 05:58:21,238 ERROR Download incomplete, downloaded 5501952 byte(s) out of 73958400
2024-01-21 05:58:21,239 WARNING Sleeping 10 seconds
2024-01-21 05:58:31,256 WARNING Resuming download at byte 5501952
2024-01-21 06:01:15,131 INFO Download rate 208.3K/s                                                                    


Result(content_length=73958400,content_type=application/x-grib,location=https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data2/adaptor.mars.internal-1705773126.0350935-9805-13-9e29fb9d-f311-4830-ae7f-43c2a8c8b47c.grib)

In [23]:
import xarray as xr
#(used to handle Earth observation data which often involves multiple dimension e.g longitude, latitude, time)

In [24]:
from osgeo import gdal, osr
#(used for handling raster data and spatial references)

In [25]:
import cfgrib
#(library, which is used to open and handle GRIB files)

In [26]:
# Input GRIB file
input_grib_file = 'download.grib'
#(Variable that stores the path to the input GRIB file containing annual temperature data.)

In [28]:
# Output GeoTIFF file
output_tiff_file = 'era5_mean_annual_temperature.tif'
#(Variable that stores the desired name for the output single-band GeoTIFF file that will contain the mean temperature.)

In [30]:
# Open the GRIB file using cfgrib
ds = cfgrib.open_dataset(input_grib_file)
#(Variable that represents the opened GRIB dataset using the 'cfgrib.open_dataset' function. This dataset contains the temperature data.)

In [31]:
# Calculate annual means
ds_annual = ds.resample(time='Y').mean(dim='time')
#(calculate the mean of the 12 months era5 temperarure data)

In [32]:
# Extract latitude, longitude, and temperature data
latitude = ds_annual.latitude.values #(Variable storing the latitude values extracted from the dataset.)
longitude = ds_annual.longitude.values #(Variable storing the longitude values extracted from the dataset.)
temperature = ds_annual.t2m.values #(Variable storing the 3D array of temperature values extracted from the dataset. The dimensions are (30, 301, 341), representing time, longitude, and latitude, respectively.)

In [33]:
# Calculate mean temperature across all years
mean_temperature = np.mean(temperature, axis=0)
#(Variable storing the calculated mean temperature by taking the mean along the time axis (axis=0) of the 3D temperature array.)

In [35]:
# Create a single-band GeoTIFF file
driver = gdal.GetDriverByName('GTiff') #(Variable storing the GDAL driver for creating a GeoTIFF file.)
tiff_dataset = driver.Create(output_tiff_file, len(longitude), len(latitude), 1, gdal.GDT_Float32)
#(Variable representing the newly created GeoTIFF dataset using the 'Create' method. It has a single band (1), and data type is set to 32-bit floating-point)

In [36]:
# Set GeoTIFF metadata
tiff_dataset.SetGeoTransform((longitude.min(), np.abs(longitude[1] - longitude[0]), 0, latitude.max(), 0, -np.abs(latitude[1] - latitude[0])))
#(Sets the geotransform parameters (spatial information) based on the minimum longitude, longitude resolution, maximum latitude, and latitude resolution.)

In [37]:
# Create a SpatialReference object and set the projection
srs = osr.SpatialReference() #(Variable representing a SpatialReference object.)
srs.ImportFromEPSG(4326)  #(Imports the WGS 84 spatial reference system using its EPSG code.)
tiff_dataset.SetProjection(srs.ExportToWkt())  #(Sets the projection of the GeoTIFF dataset using the spatial reference system.)

In [38]:
# Write mean temperature data to GeoTIFF
tiff_band = tiff_dataset.GetRasterBand(1) #(Variable representing the single band of the GeoTIFF dataset.)
tiff_band.WriteArray(mean_temperature) #(Writes the mean temperature data to the GeoTIFF band.)
tiff_band.SetNoDataValue(np.nan) #(Sets NaN as the NoData value.)
tiff_band.FlushCache()  #(Flushes the cache to ensure data is written to the file.)

In [39]:
for i in range(len(latitude)):
    tiff_band.WriteArray(mean_temperature[i:i+1, :])

In [40]:
# Close the datasets
ds.close() #(Closes the input GRIB dataset.)
tiff_dataset = None

In [41]:
print("Min Temperature:", np.nanmin(mean_temperature))
print("Max Temperature:", np.nanmax(mean_temperature))